

如何自行建立TFIDF模型

https://stackabuse.com/python-for-nlp-creating-tf-idf-model-from-scratch/

首先下載一篇文章，49 sentenses with maximum 200 words each.

解析符號化並計數


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 6 14:21:00 2019

@author: usman
"""

import nltk
nltk.download('punkt')
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

corpus = nltk.sent_tokenize(article_text)

for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


計算 idf values

In [3]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in corpus:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

計算 tf values

In [4]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in corpus:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

計算 tf-idf values

In [5]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [6]:
tf_idf_model = np.asarray(tfidf_values)

row 轉 column 變成 49 x 200

In [8]:
tf_idf_model = np.transpose(tf_idf_model)


In [10]:
tf_idf_model

array([[0.01725499, 0.00937076, 0.03249865, ..., 0.        , 0.        ,
        0.        ],
       [0.04806746, 0.06961139, 0.        , ..., 0.15231777, 0.15231777,
        0.15231777],
       [0.        , 0.04983542, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02925846, 0.01588956, 0.02755321, ..., 0.        , 0.        ,
        0.        ],
       [0.01979248, 0.        , 0.03727787, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06337238, ..., 0.        , 0.        ,
        0.        ]])

In [11]:
tf_idf_model.shape

(49, 200)